In [1]:
# libraries
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import gensim
import sklearn
import re, os, string

In [2]:
print 'Punctuation - ',string.punctuation
stop_words = set(stopwords.words('english'))
print 'Stopwords - ', stop_words

Punctuation -  !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
Stopwords -  set([u'all', u'just', u"don't", u'being', u'over', u'both', u'through', u'yourselves', u'its', u'before', u'o', u'don', u'hadn', u'herself', u'll', u'had', u'should', u'to', u'only', u'won', u'under', u'ours', u'has', u"should've", u"haven't", u'do', u'them', u'his', u'very', u"you've", u'they', u'not', u'during', u'now', u'him', u'nor', u"wasn't", u'd', u'did', u'didn', u'this', u'she', u'each', u'further', u"won't", u'where', u"mustn't", u"isn't", u'few', u'because', u"you'd", u'doing', u'some', u'hasn', u"hasn't", u'are', u'our', u'ourselves', u'out', u'what', u'for', u"needn't", u'below', u're', u'does', u"shouldn't", u'above', u'between', u'mustn', u't', u'be', u'we', u'who', u"mightn't", u"doesn't", u'were', u'here', u'shouldn', u'hers', u"aren't", u'by', u'on', u'about', u'couldn', u'of', u"wouldn't", u'against', u's', u'isn', u'or', u'own', u'into', u'yourself', u'down', u"hadn't", u'mightn', u"couldn't", u'wasn', u'y

In [3]:
text = open('Medline/text_files/doc1').read()
print 'Doc1 - \n',text

Doc1 - 
correlation between maternal and fetal plasma levels of glucose and free
fatty acids .                                                           
  correlation coefficients have been determined between the levels of   
glucose and ffa in maternal and fetal plasma collected at delivery .    
significant correlations were obtained between the maternal and fetal   
glucose levels and the maternal and fetal ffa levels . from the size of 
the correlation coefficients and the slopes of regression lines it      
appears that the fetal plasma glucose level at delivery is very strongly
dependent upon the maternal level whereas the fetal ffa level at        
delivery is only slightly dependent upon the maternal level .           


In [4]:
doc = re.sub('\r\n|\r','',text)

In [5]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

tokens = []

for token in nltk.word_tokenize(doc):
    if token not in string.punctuation and token not in stop_words:
        tokens.append(token)

print 'Comparing stemming and lemmatization\nToken\t\tStemmed Token\t\tLemmatized Token'
for token in list(set(tokens)):
    print token,'\t\t',stemmer.stem(token),'\t\t',lemmatizer.lemmatize(token)
    
print '\nLemmatization it is!'

Comparing stemming and lemmatization
Token		Stemmed Token		Lemmatized Token
correlations 		correl 		correlation
regression 		regress 		regression
ffa 		ffa 		ffa
size 		size 		size
whereas 		wherea 		whereas
collected 		collect 		collected
slopes 		slope 		slope
plasma 		plasma 		plasma
glucose 		glucos 		glucose
maternal 		matern 		maternal
dependent 		depend 		dependent
upon 		upon 		upon
delivery 		deliveri 		delivery
levels 		level 		level
significant 		signific 		significant
freefatty 		freefatti 		freefatty
appears 		appear 		appears
coefficients 		coeffici 		coefficient
obtained 		obtain 		obtained
level 		level 		level
stronglydependent 		stronglydepend 		stronglydependent
acids 		acid 		acid
lines 		line 		line
determined 		determin 		determined
correlation 		correl 		correlation
slightly 		slightli 		slightly
fetal 		fetal 		fetal

Lemmatization it is!


In [6]:
# Loading docs
path = 'Medline/text_files'
docs = {}
for subdir, dirs, files in os.walk(path):
    for file in files:
        file_path = subdir + os.path.sep + file
        doc = re.sub('\r\n|\r',' ',open(file_path, 'r').read())
        docs[file] = doc

In [7]:
docs = [ (int(re.search('[0-9]+',x).group(0)), y) for x,y in docs.items()]
docs[:2]

[(365,
  '2469. the nature of collagen disease, particularly of systemic lupus erythematosus (sle), with special reference to renal lesions (japanese) from the results of examination of 161 autopsy cases of collagen diseases, rheumatic fever and polyarteritis nodosa (pn) are considered as diseases of hyperergic nature, showing pronounced specific manifestation in particular organs.  as the reactivity becomes lower, this specific localization becomes less significant and many organs become affected in sle.  it may accordingly be supposed that sle is not a disease of hypersensitivity but one occurring in a state of exhaustion of reactability after prolonged sensitization.  moreover, a sle-like syndrome arises not only in the end stage of parasepsis, nephritis, nephrosis and some cases of prolonged sensitization by myco.tb but also in pn,pss,dm and rheumatism.  it may therefore be conceivable that sle is a syndrome rather than a separate entity.  it is most important to consider whether s

In [8]:
docs.sort(key = lambda x: x[0])
docs[:2]

[(0, ''),
 (1,
  'correlation between maternal and fetal plasma levels of glucose and free fatty acids .                                                              correlation coefficients have been determined between the levels of    glucose and ffa in maternal and fetal plasma collected at delivery .     significant correlations were obtained between the maternal and fetal    glucose levels and the maternal and fetal ffa levels . from the size of  the correlation coefficients and the slopes of regression lines it       appears that the fetal plasma glucose level at delivery is very strongly dependent upon the maternal level whereas the fetal ffa level at         delivery is only slightly dependent upon the maternal level .            ')]

In [9]:
docs = docs[1:]
docs = [y for x,y in docs]
docs[:2]

['correlation between maternal and fetal plasma levels of glucose and free fatty acids .                                                              correlation coefficients have been determined between the levels of    glucose and ffa in maternal and fetal plasma collected at delivery .     significant correlations were obtained between the maternal and fetal    glucose levels and the maternal and fetal ffa levels . from the size of  the correlation coefficients and the slopes of regression lines it       appears that the fetal plasma glucose level at delivery is very strongly dependent upon the maternal level whereas the fetal ffa level at         delivery is only slightly dependent upon the maternal level .            ',
 'changes of the nucleic acid and phospholipid levels of the livers in the course of fetal and postnatal development .                                we have followed the evolution of dna, rna and pl in the livers of rat foeti removed between the fifteenth and the 

In [10]:
# tokenize function
def tokenize(text):
    tokens = []
    for token in nltk.word_tokenize(text.lower()):
        if token not in string.punctuation and token not in stop_words:
            tokens.append(lemmatizer.lemmatize(token))
    tokens = [token for token in tokens if not token.isdigit()]
    return tokens   

In [11]:
import nltk
doc_tokens = [list(tokenize(doc_text)) for doc_text in docs]
doc_tokens[:2]

[['correlation',
  'maternal',
  'fetal',
  'plasma',
  u'level',
  'glucose',
  'free',
  'fatty',
  u'acid',
  'correlation',
  u'coefficient',
  'determined',
  u'level',
  'glucose',
  'ffa',
  'maternal',
  'fetal',
  'plasma',
  'collected',
  'delivery',
  'significant',
  u'correlation',
  'obtained',
  'maternal',
  'fetal',
  'glucose',
  u'level',
  'maternal',
  'fetal',
  'ffa',
  u'level',
  'size',
  'correlation',
  u'coefficient',
  u'slope',
  'regression',
  u'line',
  'appears',
  'fetal',
  'plasma',
  'glucose',
  'level',
  'delivery',
  'strongly',
  'dependent',
  'upon',
  'maternal',
  'level',
  'whereas',
  'fetal',
  'ffa',
  'level',
  'delivery',
  'slightly',
  'dependent',
  'upon',
  'maternal',
  'level'],
 [u'change',
  'nucleic',
  'acid',
  'phospholipid',
  u'level',
  u'liver',
  'course',
  'fetal',
  'postnatal',
  'development',
  'followed',
  'evolution',
  'dna',
  'rna',
  'pl',
  u'liver',
  'rat',
  'foeti',
  'removed',
  'fifteenth',


In [12]:
# tf-idf matrix - for clustering
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

tfidf = TfidfVectorizer(tokenizer=tokenize)
tfs = tfidf.fit_transform(docs)

counter = CountVectorizer(tokenizer=tokenize)
counter_ = counter.fit_transform(docs)



In [13]:
import json
json.dump(counter.vocabulary_, open('index_terms','w'))

In [14]:
len(counter.vocabulary_)

13644

In [22]:
tfidf.vocabulary_

{u'aided': 1404,
 u'h.m.i': 5847,
 u'cussed': 3625,
 u'glisson': 5669,
 u'1,2': 256,
 u'yellow': 13615,
 u'four': 5420,
 u'l-transposition': 7199,
 u'6-per-minute': 930,
 u'localized': 7492,
 u'seventy-seven': 11325,
 u'appa-': 1768,
 u'biomorphosis': 2264,
 u'derepressive-dedifferentiation': 3925,
 u'similarity': 11434,
 u'neuroeffectors': 8486,
 u'albumen': 1428,
 u'immature': 6448,
 u'kwashiorkor': 7186,
 u'dissolution': 4212,
 u'cytochrome': 3657,
 u'hypophyseoprivus': 6337,
 u'hormone': 6179,
 u'shielding': 11371,
 u'kent': 7127,
 u'cardiomegaly': 2571,
 u'regional': 10604,
 u'pigment': 9521,
 u'tumor-bearing': 12809,
 u'1-year-old': 282,
 u'leukon': 7364,
 u'fronto-occipital': 5470,
 u'vertical-sheet': 13308,
 u'experimentally': 4988,
 u'non-proportional': 8613,
 u'bringing': 2379,
 u'polyacrylamide': 9660,
 u'3-month-old': 698,
 u'differentiated': 4074,
 u'wooden': 13551,
 u'racic-duct': 10342,
 u'clotting-time': 2987,
 u'4613.': 847,
 u'persisted': 9384,
 u'liation': 7376,
 u'i

In [ ]:
tfidf.vocabulary_.keys

In [23]:
print tfs

  (0, 3435)	0.2676401643093625
  (0, 7789)	0.45561141415090967
  (0, 5195)	0.45111651060993085
  (0, 9565)	0.17076170229568946
  (0, 7369)	0.36499301658324745
  (0, 5689)	0.27682412904155307
  (0, 5446)	0.0645431280994378
  (0, 5142)	0.06778768230257375
  (0, 1194)	0.050769192966264996
  (0, 3025)	0.17315638538807124
  (0, 3979)	0.05985413295296133
  (0, 5202)	0.2761643185408719
  (0, 3067)	0.09000445956572757
  (0, 3847)	0.24105738585285244
  (0, 11423)	0.050620838756254644
  (0, 8781)	0.050918994557271606
  (0, 11492)	0.064913134950086
  (0, 11515)	0.09000445956572757
  (0, 10612)	0.07755299807955797
  (0, 7419)	0.06970950494873349
  (0, 1780)	0.06012590184030526
  (0, 11874)	0.08254970976290668
  (0, 3897)	0.1628236316894799
  (0, 13092)	0.12435243024307
  (0, 13482)	0.06778768230257375
  :	:
  (1032, 2434)	0.07085542135190305
  (1032, 13626)	0.04872079923276398
  (1032, 4229)	0.07085542135190305
  (1032, 1837)	0.07085542135190305
  (1032, 1467)	0.0664725291717846
  (1032, 10155)	0.

In [24]:
print counter_

  (0, 11514)	1
  (0, 13482)	1
  (0, 13092)	2
  (0, 3897)	2
  (0, 11874)	1
  (0, 1780)	1
  (0, 7419)	1
  (0, 10612)	1
  (0, 11515)	1
  (0, 11492)	1
  (0, 8781)	1
  (0, 11423)	1
  (0, 3847)	3
  (0, 3067)	1
  (0, 5202)	3
  (0, 3979)	1
  (0, 3025)	2
  (0, 1194)	1
  (0, 5142)	1
  (0, 5446)	1
  (0, 5689)	4
  (0, 7369)	8
  (0, 9565)	3
  (0, 5195)	6
  (0, 7789)	6
  :	:
  (1032, 3285)	2
  (1032, 7297)	1
  (1032, 6203)	1
  (1032, 4070)	2
  (1032, 7837)	1
  (1032, 4368)	3
  (1032, 9927)	1
  (1032, 1370)	4
  (1032, 7465)	2
  (1032, 1483)	2
  (1032, 9252)	2
  (1032, 12473)	2
  (1032, 6587)	1
  (1032, 12518)	2
  (1032, 2295)	1
  (1032, 4178)	3
  (1032, 9730)	1
  (1032, 9196)	1
  (1032, 9306)	2
  (1032, 6586)	1
  (1032, 7386)	1
  (1032, 3772)	1
  (1032, 3993)	2
  (1032, 3486)	1
  (1032, 2746)	1


In [25]:
tfs = tfs.toarray().tolist()
counter_ = counter_.toarray()

In [26]:
doc_transactions = [[] for x in xrange(len(counter_))]
for i in xrange(counter_.shape[0]):
    for j in xrange(counter_.shape[1]):
        if counter_[i][j]:
            doc_transactions[i].append(j)
            
print doc_transactions

[[1194, 1780, 3025, 3067, 3435, 3847, 3897, 3979, 5142, 5195, 5202, 5446, 5689, 7369, 7419, 7789, 8781, 9565, 10612, 11423, 11492, 11514, 11515, 11874, 13092, 13482], [283, 435, 495, 523, 551, 704, 788, 804, 880, 937, 1102, 1194, 2273, 2665, 2746, 2762, 3206, 3335, 3486, 3654, 3716, 3772, 3993, 4264, 4620, 4896, 5102, 5195, 5239, 5352, 5353, 5366, 5367, 5630, 5811, 6087, 6291, 6306, 6586, 7369, 7386, 7390, 7472, 7474, 7543, 8523, 8712, 8715, 8768, 9061, 9355, 9477, 9544, 9737, 9755, 10424, 10431, 10458, 10615, 10686, 10706, 10820, 10948, 11722, 12444, 12471, 12844, 12870, 12883, 13185, 13455, 13541, 13621], [1231, 1234, 1288, 1484, 1590, 2669, 2720, 2906, 3173, 3326, 3327, 3845, 3847, 4178, 4901, 5100, 5195, 5262, 5333, 6087, 6195, 6218, 6795, 6923, 6955, 7237, 7295, 7436, 7787, 7877, 7982, 8136, 8156, 8408, 8825, 9054, 9196, 9236, 9306, 9477, 9730, 9898, 9987, 10082, 10728, 10756, 10864, 11167, 12086, 12089, 12325, 12623, 12624, 12852], [283, 551, 581, 704, 804, 880, 1149, 1234, 2273,

In [29]:
import json
#json.dump(doc_transactions,open('doc_transactions','w'))
#json.dump(tfs, open('tfidf_matrix','w'))
#json.dump(doc_tokens, open('doc_tokens','w'))
json.dump(counter_.tolist(),open('index_terms','w'))